In [224]:
import seaborn as sns ;sns.set(rc={'figure.figsize':(20,10)})

import warnings
warnings.filterwarnings("ignore")
from IPython.display import Latex

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
plt.rcParams ['figure.figsize']  = 25, 10
plt.rcParams ['axes.labelsize']  = 14
plt.rcParams ['xtick.labelsize'] = 12
plt.rcParams ['ytick.labelsize'] = 12
plt.rcParams ['text.color']      = 'black'
plt.rcParams ['axes.labelcolor'] = 'black'
plt.rcParams ['xtick.color']     = 'black'
plt.rcParams ['ytick.color']     = 'black'
plt.style.use('fivethirtyeight')

usage: ipykernel_launcher.py [-h] [--batch-size BATCH_SIZE]
                             [--feature-maps FEATURE_MAPS] [--lr LR]
                             [--epochs EPOCHS] [--milestones MILESTONES]
                             [--gamma GAMMA] [--cosine] [--mixup]
                             [--dropout DROPOUT] [--rotations] [--model MODEL]
                             [--preprocessing PREPROCESSING]
                             [--manifold-mixup MANIFOLD_MIXUP]
                             [--temperature TEMPERATURE] [--device DEVICE]
                             [--dataset-path DATASET_PATH]
                             [--dataset-device DATASET_DEVICE]
                             [--deterministic] [--skip-epochs SKIP_EPOCHS]
                             [--runs RUNS] [--quiet] [--dataset DATASET]
                             [--dataset-size DATASET_SIZE] [--output OUTPUT]
                             [--save-features SAVE_FEATURES]
                             [--save-model SAV

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from utils import *
import datasets
from args_notebook import args
import resnet12
import few_shot_eval

utils, datasets, eval_few_shot, 

In [3]:
loaders, input_shape, num_classes, few_shot, top_5 = datasets.get_dataset(args.dataset)

### initialize few-shot meta data
if few_shot:
    num_classes, val_classes, novel_classes, elements_per_class = num_classes
    if args.dataset.lower() == "cubfs":
        elements_val, elements_novel = elements_per_class
    else:
        elements_val, elements_novel = [elements_per_class] * val_classes, [elements_per_class] * novel_classes
    print("Dataset contains",num_classes,"base classes,",val_classes,"val classes and",novel_classes,"novel classes.")
    print("Generating runs... ", end='')
    val_runs = list(zip(*[few_shot_eval.define_runs(args.n_ways, s, args.n_queries, val_classes, elements_val) for s in args.n_shots]))
    val_run_classes, val_run_indices = val_runs[0], val_runs[1]
    novel_runs = list(zip(*[few_shot_eval.define_runs(args.n_ways, s, args.n_queries, novel_classes, elements_novel) for s in args.n_shots]))
    novel_run_classes, novel_run_indices = novel_runs[0], novel_runs[1]
    print("done.")
    few_shot_meta_data = {
        "val_run_classes" : val_run_classes,
        "val_run_indices" : val_run_indices,
        "novel_run_classes" : novel_run_classes,
        "novel_run_indices" : novel_run_indices,
        "best_val_acc" : [0] * len(args.n_shots),
        "best_val_acc_ever" : [0] * len(args.n_shots),
        "best_novel_acc" : [0] * len(args.n_shots)
    }

    


Dataset contains 64 base classes, 16 val classes and 20 novel classes.
Generating runs... done.


In [4]:
### function to create model
def create_model():
    if args.model.lower() == "resnet18":
        return resnet.ResNet18(args.feature_maps, input_shape, num_classes, few_shot, args.rotations).to(args.device)
    if args.model.lower() == "resnet20":
        return resnet.ResNet20(args.feature_maps, input_shape, num_classes, few_shot, args.rotations).to(args.device)
    if args.model.lower() == "wideresnet":
        return wideresnet.WideResNet(args.feature_maps, input_shape, few_shot, args.rotations, num_classes = num_classes).to(args.device)
    if args.model.lower() == "resnet12":
        return resnet12.ResNet12(args.feature_maps, input_shape, num_classes, few_shot, args.rotations).to(args.device)
    if args.model.lower()[:3] == "mlp":
        return mlp.MLP(args.feature_maps, int(args.model[3:]), input_shape, num_classes, args.rotations, few_shot).to(args.device)
    if args.model.lower() == "s2m2r":
        return s2m2.S2M2R(args.feature_maps, input_shape, args.rotations, num_classes = num_classes).to(args.device)

model = create_model()

In [89]:

mode = 'standard'
shot = 5
train_classes = 64
out_maps = 640
feature_maps = 64

features_path = '../../../Experiments/features/'
features = torch.load(f'{features_path}/features_{mode}{shot}')

train_features = features[:64]
val_features = features[64:-16]
novel_features = features[-16:]

In [90]:
train_features = sphering(train_features)

In [91]:
train_features = train_features.reshape(600*train_classes, 1, 640)

In [203]:
from torch import nn
class transform(nn.Module):
    def __init__(self, feature_maps=64, out_maps=640):
        super(transform, self).__init__()
        self.linear = nn.Linear(10*feature_maps, out_maps, bias=True)
    
    def forward(self, x):
        return self.linear(x)
    


T = transform(feature_maps, out_maps).cuda()
L = nn.Linear(out_maps, train_classes, bias=False).cuda()




In [160]:
# train classifier on train features
crit = torch.nn.CrossEntropyLoss()

output = T(train_features)
output = sphering(output)
output = L(output)
loss = crit(output,target)

NameError: name 'target' is not defined

In [93]:
output = T(train_features)

In [98]:
output = sphering(output)

In [102]:
output = L(output)

In [221]:
[i for i in L.parameters()][1]

Parameter containing:
tensor([[-0.0080, -0.0085, -0.0382,  ..., -0.0379,  0.0293,  0.0099],
        [ 0.0351,  0.0038,  0.0221,  ...,  0.0101, -0.0386, -0.0312],
        [ 0.0331, -0.0263,  0.0039,  ...,  0.0238, -0.0336, -0.0368],
        ...,
        [ 0.0142,  0.0383, -0.0068,  ...,  0.0126,  0.0120, -0.0088],
        [-0.0329, -0.0051,  0.0358,  ...,  0.0068, -0.0196,  0.0006],
        [-0.0174,  0.0394, -0.0007,  ..., -0.0011, -0.0362, -0.0120]],
       device='cuda:0', requires_grad=True)

In [216]:
L.parameters()

<generator object Module.parameters at 0x7f90d65e6ba0>

In [200]:
torch.norm([i for i in L.parameters()][0], dim=1)

tensor([0.5770, 0.5749, 0.5799, 0.5832, 0.5788, 0.5737, 0.5815, 0.5704, 0.5632,
        0.5954, 0.5791, 0.5564, 0.5872, 0.5679, 0.5806, 0.5727, 0.5717, 0.5760,
        0.6053, 0.5750, 0.5705, 0.5687, 0.5898, 0.5745, 0.5827, 0.5797, 0.5757,
        0.5867, 0.5792, 0.5600, 0.5707, 0.5914, 0.5628, 0.5794, 0.5758, 0.5790,
        0.5896, 0.5733, 0.5807, 0.5777, 0.5918, 0.5764, 0.5708, 0.6003, 0.5641,
        0.5936, 0.5746, 0.5796, 0.5841, 0.5755, 0.5761, 0.5710, 0.5700, 0.5812,
        0.5609, 0.5930, 0.5870, 0.5818, 0.5716, 0.5860, 0.5845, 0.5850, 0.5796,
        0.5569], device='cuda:0', grad_fn=<CopyBackwards>)

In [219]:
L.weight_g.requires_grad = False

In [208]:
L = nn.utils.weight_norm(L, name='weight')

RuntimeError: Cannot register two weight_norm hooks on the same parameter weight

In [206]:
optimizer_L = torch.optim.Adam(L.weight_v, lr = 0.01)

TypeError: params argument given to the optimizer should be an iterable of Tensors or dicts, but got torch.cuda.FloatTensor

In [190]:
L.weight_g = torch.nn.Parameter(torch.ones(1, 64, device='cuda:0'))

In [191]:
[i for i in L.parameters()][0]

Parameter containing:
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], device='cuda:0',
       requires_grad=True)

In [149]:
torch.norm([i for i in L.parameters()][0], dim=1)

tensor([0.5724, 0.5843, 0.5913, 0.5689, 0.5804, 0.5829, 0.5667, 0.5531, 0.5656,
        0.6038, 0.5856, 0.5857, 0.5838, 0.5885, 0.5931, 0.6008, 0.5800, 0.6014,
        0.5472, 0.5839, 0.5864, 0.5685, 0.5583, 0.5773, 0.5777, 0.5778, 0.5838,
        0.5553, 0.5672, 0.5948, 0.5780, 0.5763, 0.5709, 0.5811, 0.5955, 0.5867,
        0.5879, 0.5709, 0.5851, 0.5891, 0.5580, 0.5852, 0.5657, 0.5671, 0.5711,
        0.5832, 0.5720, 0.5835, 0.5831, 0.5744, 0.5750, 0.5799, 0.5853, 0.5758,
        0.5704, 0.5693, 0.5674, 0.5798, 0.5633, 0.5671, 0.5833, 0.5675, 0.5688,
        0.5613], device='cuda:0', grad_fn=<CopyBackwards>)

In [175]:
torch.nn.utils.parameters_to_vector(m)

In [155]:
[i for i in L.parameters()][0]

Parameter containing:
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], requires_grad=True)

In [ ]:
torch.norm([i for i in L.parameters()][0], dim=1)

In [227]:
x = sphering(output)

In [238]:
train_features.reshape(64, 600, 640).shape

torch.Size([64, 600, 640])

In [134]:
torch.norm(m.weight_g, p = 2, dim = 0, keepdim = True)

tensor([[1.]], device='cuda:0', grad_fn=<NormBackward1>)

In [ ]:
for batch_idx, (data, target) in enumerate(train_loader):
    # encode using backbone 
    train_features = model(data)
    
    output = T(train_features)
    output = sphering(output)
    output = L(data)
    loss = crit(output,target)
    loss.backward()
    